In [1]:

import torchvision
import torchvision.transforms as transforms


ModuleNotFoundError: No module named 'torchvision'

In [ ]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

In [ ]:
import torch

batch_size = 10
train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=4,
                                           pin_memory=True,
                                           persistent_workers=True)
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=4,
                                          pin_memory=True,
                                          persistent_workers=True)

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl

checkpoint_callback = ModelCheckpoint(
    dirpath='./checkpoints',
    filename="classifier_{epoch:02d}",
    every_n_epochs=5,
    save_top_k=-1,
)

In [ ]:

trainer = pl.Trainer(accelerator='gpu', max_epochs=15, callbacks=[checkpoint_callback])


In [ ]:
from lab4.inception import Inception

#model = Inception(num_classes=10)
model = Inception.load_from_checkpoint('./checkpoints/classifier_epoch=09.ckpt', num_classes=10)

In [ ]:
torch.set_float32_matmul_precision('medium')

In [ ]:
trainer.fit(model, train_dataloaders=train_loader)

In [ ]:
trainer.test(model, test_loader)